Project Idea     
To identify disease pairs of inverse comorbidity using network analysis of disease-gene networks and disease function networks.    

       
Inverse Comorbidities       
Comorbidities are a common phenomenon in many diseases. It refers to the presence of multiple diseases or medical conditions in a patient, often leading to a worse state of disease. However, it has been observed that inverse comorbidities also exist. This condition – when an unexpectedly low probability of a disease occurring in people with another disease is observed – is intriguing, as it may help us gain insight into the pathogenesis of certain diseases and allow us to understand the underlying mechanisms of certain unknown disease pathways.     

The idea of the code is to find these inverse comorbidities by finding the pathways/functions lost in a particular genetic disorder and mapping their corresponding related diseases in the disease-gene network. 




In [1]:
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite

disease_gene_data = pd.read_csv('DG-AssocMiner_miner-disease-gene-edited.tsv', sep='\t')
disease_func_data = pd.read_csv('DF-Miner_miner-disease-function.tsv', sep='\t')
gene_func_data = pd.read_csv('GF-Miner_miner-gene-function.tsv', sep='\t')
disease_data = pd.read_csv('D-MeshMiner_miner-disease.tsv', sep='\t')

disease_func_data['# MESH_ID'] = disease_func_data['# MESH_ID'].apply(lambda x: x.split(':')[1])
disease_func_data['GO_ID'] = disease_func_data['GO_ID'].apply(lambda x: x.split(':')[1])


In [2]:
import json
import requests as req
disease_list = disease_gene_data['Disease Name'].unique()

def get_mesh_id(disease):
    url = 'id.nlm.nih.gov/mesh'
    res = req.get(f'http://{url}/lookup/descriptor?label={disease}')
    try:
        mesh_id = json.loads(res.text)[0]['resource'].split('/')[-1]
    except:
        mesh_id = None
    return mesh_id

len(disease_list)


519

In [11]:
sca_id = 'D000755'
DF_network = nx.from_pandas_edgelist(disease_func_data, '# MESH_ID', 'GO_ID')
GF_network = nx.from_pandas_edgelist(gene_func_data, '# GO_ID', 'Gene')

sca_func_list = list(DF_network.neighbors(sca_id))
func_gene_list = []
for func in sca_func_list:
    func = 'GO:' + func
    try:
        func_gene_list += list(GF_network.neighbors(func))
    except:
        pass

func_gene_list = list(set(func_gene_list))
len(func_gene_list)

260

In [12]:
gene_assoc_data = pd.read_csv('gene_associations.tsv', sep='\t')
gene_assoc_data = gene_assoc_data.iloc[:, [0, 1]]
mapping = {}
for row in gene_assoc_data.iterrows():
    mapping[row[1][0]] = row[1][1]

In [13]:
disease_gene_data['Gene_Name'] = disease_gene_data['Gene ID'].apply(lambda x: mapping[x] if x in mapping else None)
disease_gene_data = disease_gene_data.dropna()
# disease_gene_data.to_csv('DG-AssocMiner_miner-disease-gene-edited.tsv', sep='\t')

In [14]:
def get_GO_IDs(disease):
    GO_list = list(DF_network.neighbors(disease))
    return GO_list

def get_genes(GO_ID):
    func = 'GO:' + GO_ID
    func_gene_list = []
    try:
        func_gene_list += list(GF_network.neighbors(func))
    except:
        pass
    return func_gene_list


In [15]:
GO_list = get_GO_IDs(sca_id)
gene_list = []
for entry in GO_list:
    gene = get_genes(entry)
    gene_list += gene

gene_list = list(set(gene_list))


In [25]:
import numpy as np
eps = 0.1
DG_network = nx.from_pandas_edgelist(disease_gene_data, 'Disease Name', 'Gene_Name')
centralities = {}
for gene in gene_list:
    try:
        centralities[gene] = nx.degree_centrality(DG_network)[gene]
    except:
        pass
#  sort according to centrality
sorted_centralities = sorted(centralities.items(), key=lambda x: x[1], reverse=True)
nodes_removed = []
diseases = nx.bipartite.sets(DG_network)[0]
init_cent = nx.degree_centrality(DG_network)
for i in range(10):
    # print(sorted_centralities[i][0])
    try:
        DG_network.remove_node(sorted_centralities[i][0])
    except:
        continue
    new_cent = nx.degree_centrality(DG_network)
    # print(new_cent)
    for disease in diseases:
        relative_change = abs(new_cent[disease] - init_cent[disease]) / init_cent[disease]
        # print(relative_change)
        if relative_change > eps:
            nodes_removed.append(disease)
            print(relative_change)
    init_cent = new_cent

print(nodes_removed)
print(len(nodes_removed))



0.11099351766106634
0.11099347097229578
0.11099345540113242
0.12488418266048973
0.12488418266048973
0.12488416732856765
['Hypoglycemia', 'Renal Insufficiency', 'Diabetic Cardiomyopathies', 'Hypoglycemia', 'Renal Insufficiency', 'Diabetic Cardiomyopathies']
6
